# Multi Agent Framework

In [1]:
import os
import sys
sys.path.append('../src')

from openai import OpenAI
from dotenv import load_dotenv
from agents import BaseAgent, Agent, MultiAgent

### LLM Preparation 

In [2]:
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

### Init Several Agents

In [25]:
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
           handoff_description = "Call this agent if you don't know how to answer the user's question or do not has access to the necessary information.")

science_agent = Agent(base_agent = BaseAgent(client),
           name = "science agent",
           instructions = "Agent who is knowledgeable about science topics and can answer questions about them.",
           handoff_description = "Call this agent if a user is asking about a science topic like physics, chemistry, biology, etc.")

music_agent = Agent(base_agent = BaseAgent(client),
           name = "music agent",
           instructions = "Agent who is knowledgeable about music topics and can answer questions about them.",
           handoff_description = "Call this agent if a user is asking about a music topic like music theory, music history, music genres, etc.")

daily_agent = Agent(base_agent = BaseAgent(client),
           name = "daily agent",
           instructions = "Agent who is knowledgeable about daily topics and can answer questions about them.",
           handoff_description = "Call this agent if a user is asking about a daily topic like weather, news, etc.")

###  Build a Multi-Agent environment & add handoff relationships 

In [26]:
# Build a Multi-Agent environment & add handoff relationships
ma = MultiAgent(start_agent=general_agent)
ma.add_handoff_relations(from_agent=general_agent,to_agents=[science_agent,music_agent,daily_agent])
ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=daily_agent,to_agents=[general_agent])
# ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent,music_agent,daily_agent])
# ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent,science_agent,daily_agent])
# ma.add_handoff_relations(from_agent=daily_agent,to_agents=[general_agent,science_agent,music_agent])



### Multi-Agent handoff example

use handoff to get the agent who is suitable for the task

Then call the agent's pre-designed capabilities, such as streaming, tools, memory, reasoning, etc.

In [27]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue"}],agent=general_agent)

{'science_agent': <agents.Agent at 0x116a1ed90>}

In [28]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

{'science_agent': <agents.Agent at 0x116a1ed90>,
 'music_agent': <agents.Agent at 0x116a8e1d0>}

### Multi-Agent dialogue example

In [29]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue"}])

[{'role': 'assistant',
  'content': "The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of different colors, each with different wavelengths. Blue light has a shorter wavelength and is scattered in all directions by the gases and particles in the atmosphere. \n\nDuring the day, when the sun is high in the sky, more blue light is scattered than other colors, making the sky appear predominantly blue to our eyes. At sunrise and sunset, the sun's light passes through a larger portion of the atmosphere, which scatters shorter wavelengths and allows the longer wavelengths like red and orange to become more prominent, resulting in beautiful colors during those times.",
  'sender': 'science_agent'}]

In [30]:
ma.chat(messages=[{"role": "user", "content": "who are you"}])

[{'role': 'assistant',
  'content': "I am an AI trained to answer questions about science topics. I'm here to help with any inquiries you may have related to science!",
  'sender': 'science_agent'}]

In [32]:
ma.chat(messages=[{"role": "user", "content": "recommend me some music"}])

[{'role': 'assistant',
  'content': 'What genre of music are you interested in? Or do you have any specific artists or styles in mind?',
  'sender': 'music_agent'}]

In [33]:
ma.chat(messages=[{"role": "user", "content": "dinner tonight"}])

[{'role': 'assistant',
  'content': "Could you please clarify what you're looking for regarding dinner tonight? Are you seeking ideas for recipes, restaurant recommendations, or something else?",
  'sender': 'music_agent'}]

In [34]:
ma.chat(messages=[{"role": "user", "content": "who are you"}],agent=science_agent)

[{'role': 'assistant',
  'content': "I'm an AI designed to assist you with questions and topics related to science. Whether you're curious about biology, chemistry, physics, or any other scientific field, feel free to ask!",
  'sender': 'science_agent'}]

In [35]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

[{'role': 'assistant',
  'content': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. This occurs when sunlight enters Earth's atmosphere and interacts with molecules and small particles in the air.\n\nSunlight, or white light, is made up of many colors, each with different wavelengths. Blue light has a shorter wavelength compared to colors like red and orange. When sunlight passes through the atmosphere, the shorter wavelengths of light (blue and violet) are scattered in all directions by the gases and particles.\n\nAlthough both blue and violet light are scattered, our eyes are more sensitive to blue light, and some of the violet light is absorbed by the ozone layer. As a result, we predominantly see the sky as blue during the day. \n\nThe intensity of the blue color can vary based on factors such as the angle of the sun, atmospheric conditions, and the presence of particles or pollutants in the air, which can scatter light differently and sometimes crea

### Multi-Agent tool use example

In [36]:
def get_weather(city:str)->str:
    """ 
    Get the weather for a specified city.
    """
    return f"The weather in {city} is sunny."


def get_today_news()->str:
    """ 
    Get today's news.
    """
    return "ZJun created an Agent Framework."

In [37]:
daily_agent.add_tools([get_weather,get_today_news])

In [38]:
ma.chat(messages=[{"role": "user", "content": "what is the weather in Hangzhou?"}])

[{'role': 'assistant',
  'content': 'The weather in Hangzhou is currently sunny. If you need more specific details such as temperature or forecast, just let me know!',
  'sender': 'daily_agent'}]

In [39]:
ma.chat(messages=[{"role": "user", "content": "what happened today?"}])

[{'role': 'assistant',
  'content': "Today, there was a notable development as ZJun created an Agent Framework. If you have specific topics or areas of interest you'd like more details on, feel free to ask!",
  'sender': 'daily_agent'}]

In [40]:
ma.chat(messages=[{"role": "user", "content": "what is the weather in Hangzhou and what happened today?"}],
        agent=general_agent,
        show_details=False)

[{'role': 'assistant',
  'content': 'Today, ZJun announced the creation of an Agent Framework. This new development could have implications for various applications involving AI and automated systems. If you’re interested in more details or specific aspects of this news, feel free to ask!',
  'sender': 'daily_agent'},
 {'role': 'assistant',
  'content': 'The weather in Hangzhou is currently sunny. If you need more specific information like temperature or forecast, let me know!',
  'sender': 'daily_agent'}]